In [ ]:
### System Requirement ###
# At least 32 GB RAM (Cpu)

# Set-up the necessary data for the experiments in playground.ipynb

In [4]:
dataset = "imagenet"
model ="ViT-L-14-336"## Parameters
device = 'cuda'
model_name = 'ViT-L-14-336' # 'ViT-H-14'
seed = 0
num_last_layers_ = 4
subset_dim = 10
tot_samples_per_class = 50
dataset_text_name = "top_1500_nouns_5_sentences_imagenet_clean"
datataset_image_name = "imagenet"
algorithm = "svd_data_approx"
path = './datasets'

if model_name == "ViT-H-14":
    pretrained = "laion2B-s32B-b79K"
elif model_name == "ViT-L-14":
    pretrained = "laion2B-s32B-b82K"
elif model_name == "ViT-B-16":
    pretrained = "laion2B-s34B-b88K"
elif model_name == "ViT-B-32":
    pretrained = "laion2B-s34B-b79K"
elif model_name == "ViT-L-14-336":
    pretrained = "openai"


In [6]:
# Retrieve the activations (hooks) for each head in each layer, of a given ViT-model.
# Opt.: Setup a seed and use only a subset of the dataset (Imagenet)
!python -m utils.scripts.compute_activation_values --dataset {dataset} --device {device} --model {model_name} --pretrained {pretrained} --seed {seed} --batch_size 1  --samples_per_class 10 --tot_samples_per_class 50 --quantization float16 --cache_dir "../cache/"  

Model parameters: 427,944,193
Context length: 77
Vocab size: 49408
Len of res: 24
/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
We are using a dataset containing 10000 images.
  0%|                                                 | 0/10000 [00:00<?, ?it/s]palle
palle
  0%|                                      | 1/10000 [00:06<18:50:32,  6.78s/it]palle
palle
  0%|                                      | 2/10000 [00:11<15:13:08,  5.48s/it]palle
palle
  0%|                                      | 3/10000 [00:15<13:53:0

In [17]:
# Use the previous outputs (i.e. all the activations) to derive the final clip embeddings for the image dataset.
!python -m utils.scripts.compute_images_embedding --dataset {dataset} --model {model_name} --seed {seed}

Traceback (most recent call last):
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024-active/utils/scripts/compute_images_embedding.py", line 63, in <module>
    main(args)
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024-active/utils/scripts/compute_images_embedding.py", line 44, in main
    attns = np.load(os.path.join(args.output_dir, f"{args.dataset}_attn_{args.model}_seed_{args.seed}.npy"), mmap_mode="r")  # [b, l, h, d]
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/numpy/lib/npyio.py", line 405, in load
    fid = stack.enter_context(open(os_fspath(file), "rb"))
FileNotFoundError: [Errno 2]

In [18]:
# Derive the CLIP-embeddings of the classes' labels of a given dataset (i.e. allow zero-shot classifications)
!python -m utils.scripts.compute_classes_embeddings --dataset {dataset} --device {device} --model {model_name} --pretrained {pretrained} --cache_dir "../cache/"

Model parameters: 427,944,193
Context length: 77
Vocab size: 49408
100%|███████████████████████████████████████| 1000/1000 [00:25<00:00, 39.90it/s]


In [19]:
# Given a dataset of text compute their CLIP's embeddings.
!python -m utils.scripts.compute_text_embeddings --device {device} --model {model_name} --pretrained {pretrained} --cache_dir "../cache/" --data_path utils/text_descriptions/top_1500_nouns_5_sentences_imagenet_bias_clean.txt

Model parameters: 427,944,193
Context length: 77
Vocab size: 49408
100%|█████████████████████████████████████████████| 8/8 [00:06<00:00,  1.33it/s]


# Test text explanations

In [ ]:
# Test an algorithm to explain the CLIP-embeddings with text (ours: svd_data_approx, their: text_span)
!python -m utils.scripts.compute_text_explanations --device {device} --model {model_name} --algorithm svd_data_approx --dataset {dataset} --seed {seed} --num_of_last_layers 4 --text_descriptions top_1500_nouns_5_sentences_imagenet_clean

In [ ]:
# Print the components with have generated
import json

# Read JSON lines
with open("output_dir/CIFAR10_completeness_top_1500_nouns_5_sentences_imagenet_clean_ViT-B-32_algo_svd_data_approx_seed_0_max_text80.jsonl", "r") as json_file:
    for line in json_file:
        entry = json.loads(line)  # Parse each line as a JSON object
        layer = entry["layer"]
        head = entry["head"]
        texts = entry["embeddings_sort"]

        if entry["head"] == -1:
            print(entry.keys())
            print(entry["accuracy"])
        
        print(len(entry["s"]))
        print(f"Layer: {layer}, Head: {head}")
        print("Texts:")
        for text in texts:
            print(text)

In [ ]:
# Go on playground to analyze the results
models = ["ViT-B-32", "ViT-B-16", "ViT-L-14", "ViT-H-14"]
algorithms = ["svd_data_approx", "text_span"]
seed = 0
for model in models:
    for algorithm in algorithms:
        with open(f"output_dir/imagenet_completeness_top_1500_nouns_5_sentences_imagenet_clean_ViT-B-32_algo_text_span_seed_0.jsonl", "r") as json_file:

            for line in json_file:
                entry = json.loads(line)  # Parse each line as a JSON object
                layer = entry["layer"]
                head = entry["head"]
                texts = entry["embeddings_sort"]

                if entry["head"] == -1:
                    print(entry.keys())
                    print(entry["accuracy"])